# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [79]:
import pandas as pd
url_ins = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url_ins)
display(df.head())

#1 Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.
df_claims = df[(df.total_claim_amount > 1000) & (df.response == "Yes")]
display(df_claims)


#2 Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.
#3 Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.
#4 Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
#5 The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.
#6 Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,...,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A,1
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,...,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A,1
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2011-02-13,Employed,F,...,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A,2
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2011-02-13,Disabled,F,...,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A,2
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,2011-01-26,Unemployed,M,...,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,10351,FN44127,Oregon,3508.569533,Yes,Extended,College,2011-01-05,Medical Leave,M,...,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,A,1
10373,10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2011-02-08,Employed,M,...,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize,A,2
10487,10487,IX60941,Oregon,3508.569533,Yes,Extended,College,2011-01-05,Medical Leave,M,...,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,A,1
10565,10565,QO62792,Oregon,7840.165778,Yes,Extended,College,2011-01-14,Employed,M,...,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,Four-Door Car,Medsize,A,1


In [95]:
print(filtered_customers['policy_type'].unique())
print(filtered_customers['gender'].unique())


#2 Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.
#avg_claim = filtered_customers.groupby(["policy_type","gender"])["total_claim_amount"].agg("mean")
avg_claim = df[df['response'] == 'Yes'].groupby(["policy_type","gender"])["total_claim_amount"].agg("mean")
print(avg_claim)


['Corporate Auto' 'Personal Auto' 'Special Auto']
['M' 'F']
policy_type     gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: total_claim_amount, dtype: float64


In [99]:
#3 Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.
state_count = df["state"].value_counts()
print(state_count)

state
California    4183
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


In [101]:
filter_state_count = state_count[state_count > 500]
print(filter_state_count)

state
California    4183
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


In [103]:
#4 Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
stats_customer_value = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median'])
print(stats_customer_value)

                                     max          min       median
education            gender                                       
Bachelor             F       73225.95652  1904.000852  5640.505303
                     M       67907.27050  1898.007675  5548.031892
College              F       61850.18803  1898.683686  5623.611187
                     M       61134.68307  1918.119700  6005.847375
Doctor               F       44856.11397  2395.570000  5332.462694
                     M       32677.34284  2267.604038  5577.669457
High School or Below F       55277.44589  2144.921535  6039.553187
                     M       83325.38119  1940.981221  6286.731006
Master               F       51016.06704  2417.777032  5729.855012
                     M       50568.25912  2272.307310  5579.099207


In [111]:
#5 The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])
df['month'] = df['effective_to_date'].dt.month


policies = df.pivot_table(index="state",columns = "month",values = "number_of_policies", aggfunc="sum")

display(policies)

month,1,2
state,,
Arizona,3052,2864
California,6666,5901
Nevada,1493,1278
Oregon,4697,3969
Washington,1358,1225


In [113]:
#6. Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.
top_states = policies.sum(axis=1).nlargest(3).index
filtered_data = df[df['state'].isin(top_states)]
new_df = filtered_data.pivot_table(index='state', columns='month', values='number_of_policies', aggfunc='sum')
print(new_df)


month          1     2
state                 
Arizona     3052  2864
California  6666  5901
Oregon      4697  3969
